# Grover's Search algorithm: Explanation and implementation in pyQuil
## Warsaw Quantum Computing meeting

In [ ]:
import numpy as np

from pyquil import Program, get_qc
from pyquil.gates import H, I
from pyquil.api import WavefunctionSimulator

### In this example we will search a string "10"

In [ ]:
SEARCHED_STRING = "10"
N = len(SEARCHED_STRING)

### Construct quantum oracle (not a part of algorithm)

In [ ]:
oracle = np.zeros(shape=(2 ** N, 2 ** N))
for b in range(2 ** N):
    if np.binary_repr(b, N) == SEARCHED_STRING:
        oracle[b, b] = -1
    else:
        oracle[b, b] = 1
print(oracle)

## Grover's Search Algorithm
### Quantum Computer

In [ ]:
qc = get_qc(str(N)+'q', as_qvm=True)
gr_prog = Program()

### $|\psi_0 \rangle$: Qubits initilization

In [ ]:
qubits = list(reversed(qc.qubits()))
gr_prog.inst([I(q) for q in qubits])
print(WavefunctionSimulator().wavefunction(gr_prog))

### $|\psi_1 \rangle$: Apply Hadamard gates

In [ ]:
gr_prog.inst([H(q) for q in qubits])
print(WavefunctionSimulator().wavefunction(gr_prog))

### Define quantum oracle

In [ ]:
ORACLE_GATE_NAME = "GROVER_ORACLE"
gr_prog.defgate(ORACLE_GATE_NAME, oracle)

### Define inversion around the mean

In [ ]:
DIFFUSION_GATE_NAME = "DIFFUSION"
diffusion = 2.0 * np.full((2**N, 2**N), 1/(2**N)) - np.eye(2**N)
gr_prog.defgate(DIFFUSION_GATE_NAME, diffusion)

### Number of algorithm iterations

In [ ]:
N_ITER = int(np.pi / 4 * np.sqrt(2**N))
print(N_ITER)

### $|\psi_3 \rangle$: Apply Grover Operators

In [ ]:
# Loop
for i in range(N_ITER):
    
    # \psi_2^i:  Apply Quantum Oracle
    gr_prog.inst(tuple([ORACLE_GATE_NAME] + qubits))
    print(WavefunctionSimulator().wavefunction(gr_prog))
    
    # \psi_3^i:  Apply Inversion around the mean
    gr_prog.inst(tuple([DIFFUSION_GATE_NAME] + qubits))
    print(WavefunctionSimulator().wavefunction(gr_prog))

### Measure

In [ ]:
ro = gr_prog.declare('ro', 'BIT', N) # Classical registry storing the results
for i, q in enumerate(qubits):
    gr_prog.measure(q, ro[i])

### Compile and run

In [ ]:
prog_exec = qc.compile(gr_prog)
ret = qc.run(prog_exec)
ret_string = ''.join([str(q) for q in ret[0]])
print("The searched string is: {}".format(ret_string))

### Visualization tools

In [ ]:
import itertools
import numpy as np  
import matplotlib.pyplot as plt
%matplotlib inline

def plot_state_histogram(ret):
    n_trials, n_qubits = ret.shape
    # Get all combinatations of n_qubits state and assign zero prob to them
    all_states = {state: 0 for state in reversed(list(itertools.product([0, 1], repeat=n_qubits)))}
    # Calculate frequencies for measured state
    unique, counts = np.unique(ret, return_counts=True, axis=0)  
    measured_states = {tuple(state): counts[i] / n_trials for i, state in enumerate(unique)}
    # Update with frequencies
    all_states.update(measured_states)
    # Labels
    labels = ['| ' + ''.join([str(i) for i in state]) + ' >' for state in all_states.keys()]
    # Plot
    n = len(all_states.keys())
    plt.barh(range(n), list(all_states.values()), tick_label=labels)
    plt.show()

### Compile and run (multiple times)

In [ ]:
gr_prog.wrap_in_numshots_loop(100)
prog_exec = qc.compile(gr_prog)
ret = qc.run(prog_exec)
plot_state_histogram(ret)